In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM, Flatten, Conv1D, LocallyConnected1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from math import sqrt
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from matplotlib import pyplot
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
import numpy as np
import argparse
import os
from keras.callbacks import CSVLogger
import keras.backend as K


Using TensorFlow backend.


## DATA PREP

In [2]:
import data_preparation
#Convert letters to integers
integer_encoded, params = data_preparation.process_data('input.txt')

15072434
{'G': 0, 'C': 1, 'A': 2, 'T': 3}
{0: 'G', 1: 'C', 2: 'A', 3: 'T'}
[[0]
 [1]
 [1]
 [3]
 [2]
 [2]
 [0]
 [1]
 [1]
 [3]]
GCCTAAGCCT


In [8]:
print(Y_raw.shape)
print(Y_raw[0,0])
print(X[1,:])

NameError: name 'Y_raw' is not defined

In [3]:
bs=128
time_steps=64
num_epochs=20

In [4]:
#Generate traning data and its lable 
X,Y_raw, Y = data_preparation.generate_single_output_data(integer_encoded[: bs*100 + 95],bs, time_steps)


(12895, 1)
(12895,)
12800
(12800, 65)


C:\Users\quoca\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Trainning

In [5]:
def custom_loss(y_true, y_pred):
        return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)        

In [12]:
alphabet_size = Y.shape[1]
DZ_model = Sequential()
DZ_model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(Flatten())
DZ_model.add(Dense(64, activation='relu'))
DZ_model.add(Dense(alphabet_size, activation='softmax'))
optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
DZ_model.compile(loss=custom_loss, optimizer=optim)
early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
callbacks_list = [early_stopping]
DZ_model.fit(X, Y, epochs=num_epochs, batch_size=bs, verbose=1, shuffle=True, callbacks=callbacks_list)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20


KeyboardInterrupt: 

## Compression

In [ ]:
DZ_model = load_model('my_model.h5', compile=False)

In [6]:
class ArithmeticEncoder(object):
    def __init__(self, bitlen):
        self.bit_prec = bitlen   #bit precision 
        self.max_range = 1 << self.bit_prec  #max range based on bit precision 2^bit_prec
        self.mask = self.max_range - 1  #max range index starting at 0
        self.renorm= self.max_range >> 1  #renormalization threshold
        self.second_mask = self.max_range >> 1
        self.low = 0  #initial low
        self.high = self.mask  #initial high
        self.s = 0  

    def update(self, sym, c):
        low = self.low   
        high = self.high
        range_t = high - low + 1
        total = (c[-1])  #cumulative propabilities
        symlow = c[sym]  
        symhigh = c[sym+1]  
        

        newlow = low + symlow*range_t // total  #low in arithmetic integer
        newhigh = low + symhigh*range_t // total -1 #high in arithemtic integer
        self.low = int(newlow)
        self.high = int(newhigh)
        range_t = self.high - self.low 
#         print("sym:", sym)
#         print("symlow:", self.low)
#         print("symhigh:", self.high)
#         print("range:", range)
        
        #renormalization
        while((self.low ^ self.high) & self.renorm) == 0:
            self.low = (self.low << 1) 
            range_t = range_t << 1 | 1
            self.high = self.low + range_t
            self.s = self.s + 1          
#         print("renorm low:",self.low)
#         print("renorm range:",range)
#         print("renorm high:", self.high)
   
    def write(self, c, sym):
        self.update(c,sym)
        return [self.low, self.s]
        
    def finish(self):
        self.output.write(1)


# In[25]:


class ArithmeticDecoder(ArithmeticEncoder):
    def __init__(self,statesize,bitin):
        self.input = bitin
        self.bitstream = ([int(d) for d in str(self.input)])
#         print(self.bitstream)
        self.code = 0
        self.max_range = 1 << statesize
        self.renorm= self.max_range >> 1
        self.mask = self.max_range - 1
        
        self.stream = self.input[0:statesize]
        self.low = int(self.stream,2)
        self.t = statesize
        self.thresh = []
        self.thresh = [round(c[i]*self.mask) for i in range(len(c))]
        
    def decode(self):
        for i in range(len(self.thresh)-1):
            if((self.low < self.thresh[i+1]) & (self.low >= self.thresh[i])):
                sym = i
#                 print("encoded value:",self.low)
#                 print("threshold:", self.thresh)
#                 print("decoded value:", sym)
                rangenew = int(self.thresh[i+1] - self.thresh[i])
#                 print("range:", rangenew)
#                 print("low:", self.thresh[i])
                while (rangenew < self.renorm):
                    rangenew = rangenew << 1 
                    self.low = self.low << 1
                    self.thresh[i] = int(self.thresh[i]) << 1
#                     print("renorm val;",self.low)
#                     print("renorm range;",rangenew)
                    self.thresh[i+1] = rangenew + self.thresh[i]
#                     print("renorm low:" ,self.thresh[i])
                self.thresh = [round(c[j]*(self.thresh[i+1]-self.thresh[i]))+self.thresh[i] for j in range(len(c))]
                break
        return sym

In [7]:
alphabet_size = 4
def cumul_d(prob):
    return np.cumsum(prob, axis = 1)  
import arithmetic_coder

symbol stream input: ['a' 'c' 'g' 't' 't' 'a' 'c' 't' 'g' 'a' 't' 'a' 'a' 't' 'a' 'c' 'c' 'g'
 't']
num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3]
probability: [0.5 0.2 0.2 0.1]
cumulative prob: [0, 0.5, 0.7, 0.8999999999999999, 0.9999999999999999]
encoded bit length: 106
decoded num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3, 1]
decoded symbol stream: ['a', 'c', 'g', 't', 't', 'a', 'c', 't', 'g', 'a', 't', 'a', 'a', 't', 'a', 'c', 'c', 'g', 't']


In [12]:
bitprecision = 8
l = int(X.shape[0] / bs) * bs 
print(l)
print((X.shape[0])-time_steps)

#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1, dtype = float)
c[1:] = np.cumsum(prob)        
print(c - 0.0000001)
bitstream = []
enc = ArithmeticEncoder(bitprecision)

num_iters = int((len(X)+time_steps)/bs)
ind = np.array(range(bs))*num_iters
prob = DZ_model.predict(X[ind,:], batch_size=bs)
print(num_iters)

#Encoding
for i in (range(bs)):
    for j in range(min(time_steps, num_iters)):
        new = enc.write( X[ind[i], j], c)
        
low_final = new[0] #fina low value 
s_final = new[1]  #count of renormalizations
ind = ind + 1
#convert low to bitstream, ensure proper number of bits 
bitstream_n = format(int(low_final), 'b')  
cumul = np.zeros((bs, alphabet_size+1))        
for k in (range(num_iters - time_steps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    new = enc.write( X[ind,:], cumul[i,:].tolist())
    for i in range(bs):
        print('b')
        enc.write(Y_raw[ind[i]], cumul[i,:].tolist())
    ind = ind + 1
    
    
    
    
    
low_final = new[0] #fina low value 
s_final = new[1]  #count of renormalizations
ind = ind + 1
#convert low to bitstream, ensure proper number of bits 
bitstream_n = format(int(low_final), 'b')  
while len(bitstream_n) < (s_final+bitprecision):
    bitstream = '0' + bitstream_n
print("encoded bit length:", len(bitstream))
   
    
# else:
#     prob = np.ones(alphabet_size)/alphabet_size
#     cumul = np.zeros(alphabet_size+1, dtype = float)
#     cumul[1:] = np.cumsum(prob)         
#     print(cumul.tolist())
#     enc = ArithmeticEncoder(bitprecision)
#     for j in range(time_steps):
#         new = enc.write( X[0,j], cumul.tolist())
#     for i in (range(len(X))):
#         prob = DZ_model.predict(X[i,:].reshape(1,-1), batch_size=1)
#         cumul[1:] = np.cumsum(prob)
#         new = enc.write( Y_raw[i][0], cumul)
#     #convert low to bitstream, ensure proper number of bits 
#     bitstream = format(int(low_final), 'b')  
#     while len(bitstream) < (s_final+bitprecision):
#         bitstream = '0' + bitstream
#     print("encoded bit length:", len(bitstream))   

 #START DECODER
print(bitstream)
dec = ArithmeticDecoder(bitprecision, bitstream)

symdec = []
for symbols in range(len(sym)):
    symdec.append(dec.decode())
print("decoded num stream:", symdec)




12800
12736
[-1.000000e-07  2.499999e-01  4.999999e-01  7.499999e-01  9.999999e-01]
100


OverflowError: int too large to convert to float

In [38]:
X[i,:].reshape(1,-1).shape

(1, 64)

In [37]:
if (l > (len(X.shape[0])-timesteps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    for i in range(bs):
            enc[i].write(cumul[i,:], y_original[ind[i]])
    ind = ind + 1
else:

NameError: name 'l' is not defined

In [8]:
DZ_model = load_model('my_model.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
#make an array of cumulative probabilities
c = []
for i in range(len(prob)+1):
    c.append(sum(prob[0:i]))

cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)  
for j in (range(num_iters - timesteps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    for i in range(bs):
        new = enc.write(Y_raw[ind[i]], cumul[i,:].tolist())
    ind = ind + 1
    low_final = new[0] #fina low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b')  
while len(bitstream) < (s_final+bitprecision):
    bitstream = '0' + bitstream
    print("encoded bit length:", len(bitstream))


    #START DECODER
    dec = ArithmeticDecoder(bitprecision, bitstream)

    symdec = []
    for symbols in range(len(sym)):
        symdec.append(dec.decode())
    print("decoded num stream:", symdec)

    #convert numbers back to letters
    dec_stream = []
    for x, sym in enumerate(symdec): 
        if sym == 0:
            dec_stream.append('a')
        elif sym ==1:
            dec_stream.append('c')
        elif sym == 2:
            dec_stream.append('g')
        elif sym == 3:
            dec_stream.append('t')

    dec_stream = dec_stream[:-1] 
    print("decoded symbol stream:", dec_stream)

NameError: name 'timesteps' is not defined

In [17]:
cumul

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0,

In [9]:

print(X)
print(X[ind,:].shape)

[[0 3 3 ... 3 3 1]
 [3 3 1 ... 3 1 2]
 [3 1 2 ... 1 2 2]
 ...
 [3 0 0 ... 1 1 1]
 [0 0 3 ... 1 1 3]
 [0 3 0 ... 1 3 1]]
(128, 64)


In [34]:
cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)  
print(cumul[0,:].tolist())


[0, 0, 0, 0, 0]


In [11]:
print(sum(prob[1,:]))

1.0000000596046448


In [13]:
import arithmetic_coder

symbol stream input: ['a' 'c' 'g' 't' 't' 'a' 'c' 't' 'g' 'a' 't' 'a' 'a' 't' 'a' 'c' 'c' 'g'
 't']
num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3]
probability: [0.5 0.2 0.2 0.1]
cumulative prob: [0, 0.5, 0.7, 0.8999999999999999, 0.9999999999999999]
encoded bit length: 106
decoded num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3, 1]
decoded symbol stream: ['a', 'c', 'g', 't', 't', 'a', 'c', 't', 'g', 'a', 't', 'a', 'a', 't', 'a', 'c', 'c', 'g', 't']


In [50]:
prob = np.ones(alphabet_size)/alphabet_size
cumul = np.zeros(alphabet_size+1, dtype = float)
cumul[1:] = np.cumsum(prob)        

In [51]:
cumul

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [8]:
prob = np.ones(alphabet_size)/alphabet_size
cumul = np.zeros(alphabet_size+1, dtype = np.uint64)
cumul[1:] = np.cumsum(prob*10000000 + 1)   

In [9]:
cumul

array([       0,  2500001,  5000002,  7500003, 10000004], dtype=uint64)